In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


import training and test data

In [2]:
train = pd.read_csv('train.csv',index_col = 'ID')
test = pd.read_csv('test.csv',index_col ='ID')

In [3]:
train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
ID,,,,,,,,,,,,,,,,,,,,,
0,130.81,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
6,88.53,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
7,76.26,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
9,80.62,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
13,78.02,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


If for any column(s), the variance is equal to zero, then you need to remove those variable(s).


In [4]:
variance = pow(train.drop(columns={'y'}).std(),2).to_dict()

null_cnt = 0
for key, value in variance.items():
    if(value==0):
        print('Name = ',key)
        null_cnt = null_cnt+1
print('No of columns which has zero variance = ',null_cnt)


Name =  X11
Name =  X93
Name =  X107
Name =  X233
Name =  X235
Name =  X268
Name =  X289
Name =  X290
Name =  X293
Name =  X297
Name =  X330
Name =  X347
No of columns which has zero variance =  12


In [5]:
train= train.drop(columns={'X11','X93','X107','X233','X235','X268','X289','X290','X293','X297','X330','X347'})
train.shape

(4209, 365)

Check for null and unique values for test and train sets


In [6]:
train.isnull().sum().any()

False

Apply label encoder

In [7]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()

In [8]:
for col in train.columns:
    if(train[col].dtype != np.float64 and train[col].dtype != np.int64):
        
        # making a list of unique strings in train and test feature
        unique_train = train[col].unique().tolist()
        unique_test = test[col].unique().tolist()
        
        # making a combined list
        for member in unique_test:
            if member not in unique_train:
                unique_train.append(member)
               
# mapping with numbers
        map_dict = dict(zip(unique_train, range(len(unique_train))))
        train[col] = train[col].replace(to_replace = map_dict)
        test[col] = test[col].replace(to_replace = map_dict)
        
     

In [9]:
   train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
ID,,,,,,,,,,,,,,,,,,,,,
0,130.81,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,88.53,0,1,1,1,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
7,76.26,1,2,2,2,0,2,0,1,0,...,0,0,0,0,0,0,1,0,0,0
9,80.62,1,1,2,3,0,2,1,2,0,...,0,0,0,0,0,0,0,0,0,0
13,78.02,1,0,2,3,0,3,2,3,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X_train = train.drop(columns={'y'})
Y_train = train.y

Perform dimensionality reduction

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=.95)
pca.fit(X_train,Y_train)

PCA(n_components=0.95)

In [12]:
X_train_trans = pca.fit_transform(X_train)
print(X_train_trans.shape)

(4209, 12)


In [13]:
test = test.drop(columns={'X11','X93','X107','X233','X235','X268','X289','X290','X293','X297','X330','X347'})


In [14]:
test = pca.fit_transform(test)

Predict your test_df values using XGBoost

In [15]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [16]:

grid_values = {'n_estimators': [74,75,76], 'learning_rate': [0.13,0.135,0.14], 'max_depth': [1,2,3]}
grid_xgb = GridSearchCV(xgb.XGBRegressor(), param_grid = grid_values, cv=10, scoring = 'r2')
grid_xgb.fit(X_train_trans, Y_train)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_par

In [19]:
predict_xgb = grid_xgb.predict(test)


Final Predicitions and writing it to .csv file

In [20]:
final_predictions = pd.DataFrame()
final_predictions['y'] = pd.Series(predict_xgb)
final_predictions.to_csv('predictions.csv', index=False)

In [21]:
final_predictions.head()

,y
0,79.025505
1,88.508324
2,79.602554
3,76.361130
4,96.902840
